In [1]:
print("Need to download the .nc files via NCSS at:","\n","http://thredds.ucar.edu/thredds/ncss/grib/NCEP/GFS/Global_0p25deg/GFS_Global_0p25deg_YYYMMDD_FFFF.grib2/dataset.html")
print("Example:","\n","http://thredds.ucar.edu/thredds/ncss/grib/NCEP/GFS/Global_0p25deg/GFS_Global_0p25deg_20190910_0000.grib2/dataset.html")


Need to download the .nc files via NCSS at: 
 http://thredds.ucar.edu/thredds/ncss/grib/NCEP/GFS/Global_0p25deg/GFS_Global_0p25deg_YYYMMDD_FFFF.grib2/dataset.html
Example: 
 http://thredds.ucar.edu/thredds/ncss/grib/NCEP/GFS/Global_0p25deg/GFS_Global_0p25deg_20190910_0000.grib2/dataset.html



# Cross Section Analysis


The MetPy function `metpy.interpolate.cross_section` can obtain a cross-sectional slice through
gridded data.



In [2]:
from metpy.plots import ctables
mycmap = ctables.registry.get_colortable("NWSReflectivity")

In [3]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pygrib as pg
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.interpolate import cross_section
# Importing Datetime Libraries
from datetime import datetime, timedelta
from netCDF4 import num2date
from matplotlib import patheffects
#import Hi_Lo
import glob, re

In [4]:
from copy import copy

%matplotlib inline
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import matplotlib.pyplot as plt
import numpy as np
import shapely.geometry as sgeom

In [5]:
def find_side(ls, side):
    """
    Given a shapely LineString which is assumed to be rectangular, return the
    line corresponding to a given side of the rectangle.
    
    """
    minx, miny, maxx, maxy = ls.bounds
    points = {'left': [(minx, miny), (minx, maxy)],
              'right': [(maxx, miny), (maxx, maxy)],
              'bottom': [(minx, miny), (maxx, miny)],
              'top': [(minx, maxy), (maxx, maxy)],}
    return sgeom.LineString(points[side])


def lambert_xticks(ax, ticks):
    """Draw ticks on the bottom x-axis of a Lambert Conformal projection."""
    te = lambda xy: xy[0]
    lc = lambda t, n, b: np.vstack((np.zeros(n) + t, np.linspace(b[2], b[3], n))).T
    xticks, xticklabels = _lambert_ticks(ax, ticks, 'bottom', lc, te)
    ax.xaxis.tick_bottom()
    ax.set_xticks(xticks)
    ax.set_xticklabels([ax.xaxis.get_major_formatter()(xtick) for xtick in xticklabels])
    

def lambert_yticks(ax, ticks):
    """Draw ricks on the left y-axis of a Lamber Conformal projection."""
    te = lambda xy: xy[1]
    lc = lambda t, n, b: np.vstack((np.linspace(b[0], b[1], n), np.zeros(n) + t)).T
    yticks, yticklabels = _lambert_ticks(ax, ticks, 'left', lc, te)
    ax.yaxis.tick_left()
    ax.set_yticks(yticks)
    ax.set_yticklabels([ax.yaxis.get_major_formatter()(ytick) for ytick in yticklabels])

def _lambert_ticks(ax, ticks, tick_location, line_constructor, tick_extractor):
    """Get the tick locations and labels for an axis of a Lambert Conformal projection."""
    outline_patch = sgeom.LineString(ax.outline_patch.get_path().vertices.tolist())
    axis = find_side(outline_patch, tick_location)
    n_steps = 30
    extent = ax.get_extent(ccrs.PlateCarree())
    _ticks = []
    for t in ticks:
        xy = line_constructor(t, n_steps, extent)
        proj_xyz = ax.projection.transform_points(ccrs.Geodetic(), xy[:, 0], xy[:, 1])
        xyt = proj_xyz[..., :2]
        ls = sgeom.LineString(xyt.tolist())
        locs = axis.intersection(ls)
        if not locs:
            tick = [None]
        else:
            tick = tick_extractor(locs.xy)
        _ticks.append(tick[0])
    # Remove ticks that aren't visible:    
    ticklabels = copy(ticks)
    while True:
        try:
            index = _ticks.index(None)
        except ValueError:
            break
        _ticks.pop(index)
        ticklabels.pop(index)
    return _ticks, ticklabels

In [6]:
def OmegaCross(File,Year,Month,Day,Step,start,end,Radar=False,Grids=False,inset_dims=None,Extent=None):
    # Define the figure object and primary axes
    fig = plt.figure(1, figsize=(16., 9.))
    ax = plt.axes()

    
    ##############################################################################################################
    # Set up cross section 
    ##############################################################################################################
    
    data = xr.open_dataset(File)
    #data = pg.open('GFS_Global_0p25deg_20190313_0000.grib2.nc')
    #data.read()
    data = data.metpy.parse_cf().squeeze()
    #start = (40, -97.0)
    #end = (36, -93.0)
    cross = cross_section(data, start, end)
    cross.set_coords(('lat', 'lon'), True)

    temperature, pressure = xr.broadcast(cross['Temperature_isobaric'],cross['isobaric4'])

    theta = mpcalc.potential_temperature(pressure, temperature)
    #rh = mpcalc.relative_humidity_from_specific_humidity(specific_humidity, temperature, pressure)

    # These calculations return unit arrays, so put those back into DataArrays in our Dataset
    cross['Potential_temperature'] = xr.DataArray(theta,
                                                  coords=temperature.coords,
                                                  dims=temperature.dims,
                                                  attrs={'units': theta.units})
    #cross['Relative_humidity'] = xr.DataArray(rh,
    #                                          coords=specific_humidity.coords,
    #                                          dims=specific_humidity.dims,
    #                                          attrs={'units': rh.units})

    cross['u-component_of_wind_isobaric'].metpy.convert_units('knots')
    cross['v-component_of_wind_isobaric'].metpy.convert_units('knots')
    cross['t_wind'], cross['n_wind'] = mpcalc.cross_section_components(cross['u-component_of_wind_isobaric'],
                                                                       cross['v-component_of_wind_isobaric'])
    #cross['dew'] =  xr.broadcast(cross['Geopotential_height_potential_vorticity_surface'])
    #print(cross)




    
    
    # Plot potential temperature using contour, with some custom labeling
    #theta_contour = ax.contour(lons, cross['isobaric'], cross['Potential_temperature'][0,:,:],
    #                           levels=np.arange(250, 450, 5), colors='k', linewidths=2)

    omega_contour = ax.contourf(cross['lon'], cross['isobaric6'],cross['Vertical_velocity_pressure_isobaric'][Step,:],
                             30,cmap='PuOr_r')
    cbar = plt.colorbar(omega_contour, orientation='vertical',pad=0.004)#,cax=cbaxes)


    theta_contour = ax.contour(cross['lon'], cross['isobaric4'], cross['Potential_temperature'][Step,:,:],
                               levels=np.arange(250, 850, 3), colors='k', linewidths=2)


    # Colorbar Axis Placement (under figure)
    #colorbar_axis = [0.183, 0.09, 0.659, 0.03] # [left, bottom, width, height]
    #cbaxes = fig.add_axes(colorbar_axis)


    # Plot winds using the axes interface directly, with some custom indexing to make the barbs
    # less crowded
    wind_slc_vert = list(range(0, 19, 2)) + list(range(19, 29))
    wind_slc_horz = slice(5, 100, 5)
    #ax.barbs(cross['lon'][wind_slc_horz], cross['isobaric'][wind_slc_vert],
    #         cross['t_wind'][Step,:,:][wind_slc_vert, wind_slc_horz],
    #         cross['n_wind'][Step,:,:][wind_slc_vert, wind_slc_horz], color='b')

    # Adjust the y-axis to be logarithmic
    #ax.set_yscale('symlog')
    plt.yscale('log')
    #ax.set_yticklabels(np.arange(1000, 50, -100))
    #ax.set_ylim(cross['isobaric'].max(), cross['isobaric'].min())
    ax.set_ylim(cross['isobaric'].max(), cross['isobaric'][10])



    ##############################################################################################################
    # Insert
    ##############################################################################################################

    # Define the CRS and inset axes
    data_crs = data['Relative_humidity_isobaric'].metpy.cartopy_crs
    print(data_crs)
    #4-tuple of floats *rect* = ``[left, bottom, width, height]
    if inset_dims !=None:
        ax_inset = plt.axes(inset_dims, projection=ccrs.LambertConformal())
    else:
        
        ax_inset = plt.axes((0.02, 0.58, 0.23, 0.2), projection=ccrs.LambertConformal())
    
    plot_proj = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)
    
    if Grids == True:
        # *must* call draw in order to get the axis boundary used to add ticks:
        fig.canvas.draw()

        # Define gridline locations and draw the lines using cartopy's built-in gridliner:
        xticks = [np.arange(-180,180)]
        yticks = [np.arange(0,90)]
        ax_inset.gridlines(xlocs=xticks, ylocs=yticks)

        # Label the end-points of the gridlines using the custom tick makers:
        ax_inset.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
        ax_inset.yaxis.set_major_formatter(LATITUDE_FORMATTER)
        #lambert_xticks(ax, xticks)
        #lambert_yticks(ax, yticks)

        #gl = ax_inset.gridlines(draw_labels=True,crs=ccrs.LambertConformal())
        #gl.xlabels_top = gl.ylabels_right = False
        #gl.xformatter = LONGITUDE_FORMATTER
        #gl.yformatter = LATITUDE_FORMATTER
    if Radar == True:
        from siphon.catalog import TDSCatalog
        top_cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog.xml')
        ref = top_cat.catalog_refs['Radar Data']

        new_cat = ref.follow()
        model = new_cat.catalog_refs[4]
        gfs_cat = model.follow()

        ds = gfs_cat.datasets[0]
        subset = ds.subset()
        query_data = subset.query()

        from datetime import datetime, timedelta
        now = datetime(Year,Month,Day,Step*3,0)
        query_data.accept('netcdf4')
        query_data.time(now)
        query_data.variables('Base_reflectivity_surface_layer').add_lonlat()

        radar_data = subset.get_data(query_data)

        time = radar_data.variables['time']
        lon = radar_data.variables['lon']
        lat = radar_data.variables['lat']
        dtime = num2date(time[:],time.units)
        X = radar_data.variables['x'][:]
        Y = radar_data.variables['y'][:]

        Time = dtime[0].strftime('%Y-%m-%d %H:%MZ')
        reflec_data = radar_data.variables['Base_reflectivity_surface_layer']
        reflec = radar_data.variables['Base_reflectivity_surface_layer'][:]
        reflec = np.ma.masked_where(reflec < 9,reflec)

        RADAR_levs = np.arange(0,75,1)
    
        cs = ax_inset.pcolormesh(lon[:], lat[:], reflec[0,:,:], cmap=mycmap,transform=ccrs.PlateCarree(),vmin=0, vmax=75)
        #cbar = plt.colorbar(cs, orientation='vertical',pad=0.005,aspect=50)
        text_time = ax_inset.text(.995, 0.01, 
                    Time,
                    horizontalalignment='right', transform=ax_inset.transAxes,
                    color='w', fontsize=12, weight='bold')

        text_time2 = ax_inset.text(0.005, 0.01, 
                        "NWS Radar (dbz) & 500mb Heights",
                        horizontalalignment='left', transform=ax_inset.transAxes,
                        color='w', fontsize=12, weight='bold')

        outline_effect = [patheffects.withStroke(linewidth=5, foreground='k')]
        text_time.set_path_effects(outline_effect)
        text_time2.set_path_effects(outline_effect)
        
        



    # Plot geopotential height at 500 hPa using xarray's contour wrapper
    ax_inset.contour(data['lon'], data["lat"], data['Geopotential_height_isobaric'][Step,21,:,:],
                     levels=np.arange(5100, 6000, 40), colors='k',transform=data_crs,alpha=0.8)



    #Hi_Lo.plot_maxmin_points(ax_inset,data['lon'], data['lat'], data['MSLP_Eta_model_reduction_msl']/100., 'max', 
    #                         50, symbol='H', color='b',  transform=data_crs)

    #Hi_Lo.plot_maxmin_points(ax_inset,data['lon'], data['lat'], data['MSLP_Eta_model_reduction_msl']/100., 'min', 
    #                         25, symbol='L', color='r', transform=data_crs)


    # Plot the path of the cross section
    endpoints = data_crs.transform_points(ccrs.LambertConformal(),
                    *np.vstack([start, end]))

    #print(endpoints)
    #print(cross['lon'][0],cross['lon'][-1],cross['lat'][-1],cross['lat'][0])
    
    ax_inset.scatter(cross['lon'][0],cross['lat'][0], c='r', zorder=15,transform=data_crs,marker='$S$',s=50)
    ax_inset.scatter(cross['lon'][-1],cross['lat'][-1], c='r', zorder=15,transform=data_crs,marker='$E$',s=50)
    #ax_inset.scatter(259.5,37.25, c='b', zorder=1,transform=data_crs)
    ax_inset.plot(cross['lon'], cross['lat'], c='r', zorder=2,transform=data_crs)

    # Add geographic features
    ax_inset.coastlines()
    ax_inset.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='b', alpha=0.8, zorder=0)

    #extent = [-130,-60,20,52]
    if Extent !=None:
        extent = Extent
    else:
        extent = [start[1]+10,end[1]-15,start[0]-10,end[0]+15]
    ax_inset.set_extent(extent,ccrs.PlateCarree())
    # Set the titles and axes labels
    #ax_inset.set_title('NWS Radar (dbz) and 500mb Heights')
    #ax.set_title('NARR Cross-Section \u2013 {} to {} \u2013 Valid: {}\n'
    #             'Potential Temperature (K), Tangential/Normal Winds (knots), '
    #             'Relative Humidity (dimensionless)\n'
    #             'Inset: Cross-Section Path and 500 hPa Geopotential Height'.format(
    #                 start, end, cross['time'].dt.strftime('%Y-%m-%d %H:%MZ').item()))
    ax.set_ylabel('Pressure (Pa)')
    ax.set_xlabel('Longitude (degrees east)')

    time = data.variables["time"]
    print(time.values)

    #type(time.values)
    time.values.tolist()
    time.values[Step].astype('datetime64[s]')
    TIME = time.values[Step].astype('datetime64[s]').tolist()
    #type(TIME)
    file_time = '{0:%Y_%m_%d_%H:%MZ}'.format(TIME)

    ax.set_title("GFS: Theta and Omega Cross Section "+"{0:%Y-%m-%d %H:%MZ}".format(TIME))
    fig.savefig("/Users/chowdahead/Desktop/Omega_cross"+file_time.replace(":","")+"Z.png",bbox_inches='tight',dpi=120)
    plt.close(fig)
    #plt.show()

# Relative Humidity

In [7]:
def RHCross(File,Year,Month,Day,Step,start,end,Radar=False,Grids=False,inset_dims=None,Extent=None):
    # Define the figure object and primary axes
    fig = plt.figure(1, figsize=(16., 9.))
    ax = plt.axes()

    
    ##############################################################################################################
    # Set up cross section 
    ##############################################################################################################
    
    data = xr.open_dataset(File)
    #data = pg.open('GFS_Global_0p25deg_20190313_0000.grib2.nc')
    #data.read()
    data = data.metpy.parse_cf().squeeze()
    #start = (40, -97.0)
    #end = (36, -93.0)
    cross = cross_section(data, start, end)
    cross.set_coords(('lat', 'lon'), True)

    temperature, pressure = xr.broadcast(cross['Temperature_isobaric'],cross['isobaric4'])

    theta = mpcalc.potential_temperature(pressure, temperature)
    #rh = mpcalc.relative_humidity_from_specific_humidity(specific_humidity, temperature, pressure)

    # These calculations return unit arrays, so put those back into DataArrays in our Dataset
    cross['Potential_temperature'] = xr.DataArray(theta,
                                                  coords=temperature.coords,
                                                  dims=temperature.dims,
                                                  attrs={'units': theta.units})
    #cross['Relative_humidity'] = xr.DataArray(rh,
    #                                          coords=specific_humidity.coords,
    #                                          dims=specific_humidity.dims,
    #                                          attrs={'units': rh.units})

    cross['u-component_of_wind_isobaric'].metpy.convert_units('knots')
    cross['v-component_of_wind_isobaric'].metpy.convert_units('knots')
    cross['t_wind'], cross['n_wind'] = mpcalc.cross_section_components(cross['u-component_of_wind_isobaric'],
                                                                       cross['v-component_of_wind_isobaric'])
    #cross['dew'] =  xr.broadcast(cross['Geopotential_height_potential_vorticity_surface'])
    #print(cross)


    # Plot potential temperature using contour, with some custom labeling
    #theta_contour = ax.contour(lons, cross['isobaric'], cross['Potential_temperature'][0,:,:],
    #                           levels=np.arange(250, 450, 5), colors='k', linewidths=2)

    rh_contour = ax.contourf(cross['lon'], cross['isobaric'], cross['Relative_humidity_isobaric'][Step,:],
                            cmap="BrBG",levels=np.arange(0,120,1),alpha=0.7)
    cbar = plt.colorbar(rh_contour, orientation='vertical',pad=0.004)#,cax=cbaxes)
    
    theta_contour = ax.contour(cross['lon'], cross['isobaric4'], cross['Potential_temperature'][Step,:,:],
                               levels=np.arange(250, 850, 3), colors='k', linewidths=2)


    # Colorbar Axis Placement (under figure)
    #colorbar_axis = [0.183, 0.09, 0.659, 0.03] # [left, bottom, width, height]
    #cbaxes = fig.add_axes(colorbar_axis)


    # Plot winds using the axes interface directly, with some custom indexing to make the barbs
    # less crowded
    wind_slc_vert = list(range(0, 19, 2)) + list(range(19, 29))
    wind_slc_horz = slice(5, 100, 5)
    #ax.barbs(cross['lon'][wind_slc_horz], cross['isobaric'][wind_slc_vert],
    #         cross['t_wind'][Step,:,:][wind_slc_vert, wind_slc_horz],
    #         cross['n_wind'][Step,:,:][wind_slc_vert, wind_slc_horz], color='b')

    # Adjust the y-axis to be logarithmic
    #ax.set_yscale('symlog')
    plt.yscale('log')
    #ax.set_yticklabels(np.arange(1000, 50, -100))
    #ax.set_ylim(cross['isobaric'].max(), cross['isobaric'].min())
    ax.set_ylim(cross['isobaric'].max(), cross['isobaric'][10])



    ##############################################################################################################
    # Insert
    ##############################################################################################################

    # Define the CRS and inset axes
    data_crs = data['Relative_humidity_isobaric'].metpy.cartopy_crs
    print(data_crs)
    #4-tuple of floats *rect* = ``[left, bottom, width, height]
    if inset_dims !=None:
        ax_inset = plt.axes(inset_dims, projection=ccrs.LambertConformal())
    else:
        
        ax_inset = plt.axes((0.02, 0.58, 0.23, 0.2), projection=ccrs.LambertConformal())
    #ax_inset = fig.add_axes([0.125, 0.665, 0.25, 0.25], projection=data_crs)
    
    plot_proj = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)
    if Grids == True:
        # *must* call draw in order to get the axis boundary used to add ticks:
        fig.canvas.draw()

        # Define gridline locations and draw the lines using cartopy's built-in gridliner:
        xticks = [np.arange(-180,180)]
        yticks = [np.arange(0,90)]
        ax_inset.gridlines(xlocs=xticks, ylocs=yticks)

        # Label the end-points of the gridlines using the custom tick makers:
        ax_inset.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
        ax_inset.yaxis.set_major_formatter(LATITUDE_FORMATTER)
        #lambert_xticks(ax, xticks)
        #lambert_yticks(ax, yticks)

        #gl = ax_inset.gridlines(draw_labels=True,crs=ccrs.LambertConformal())
        #gl.xlabels_top = gl.ylabels_right = False
        #gl.xformatter = LONGITUDE_FORMATTER
        #gl.yformatter = LATITUDE_FORMATTER
    if Radar == True:
        from siphon.catalog import TDSCatalog
        top_cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog.xml')
        ref = top_cat.catalog_refs['Radar Data']

        new_cat = ref.follow()
        model = new_cat.catalog_refs[4]
        gfs_cat = model.follow()

        ds = gfs_cat.datasets[0]
        subset = ds.subset()
        query_data = subset.query()

        from datetime import datetime, timedelta
        now = datetime(Year,Month,Day,Step*3,0)
        query_data.accept('netcdf4')
        query_data.time(now)
        query_data.variables('Base_reflectivity_surface_layer').add_lonlat()

        radar_data = subset.get_data(query_data)

        time = radar_data.variables['time']
        lon = radar_data.variables['lon']
        lat = radar_data.variables['lat']
        dtime = num2date(time[:],time.units)
        X = radar_data.variables['x'][:]
        Y = radar_data.variables['y'][:]

        Time = dtime[0].strftime('%Y-%m-%d %H:%MZ')
        reflec_data = radar_data.variables['Base_reflectivity_surface_layer']
        reflec = radar_data.variables['Base_reflectivity_surface_layer'][:]
        reflec = np.ma.masked_where(reflec < 9,reflec)

        RADAR_levs = np.arange(0,75,1)
    
        cs = ax_inset.pcolormesh(lon[:], lat[:], reflec[0,:,:], cmap=mycmap,transform=ccrs.PlateCarree(),vmin=0, vmax=75)
        #cbar = plt.colorbar(cs, orientation='vertical',pad=0.005,aspect=50)
        text_time = ax_inset.text(.995, 0.01, 
                    Time,
                    horizontalalignment='right', transform=ax_inset.transAxes,
                    color='w', fontsize=12, weight='bold')

        text_time2 = ax_inset.text(0.005, 0.01, 
                        "NWS Radar (dbz) & 500mb Heights",
                        horizontalalignment='left', transform=ax_inset.transAxes,
                        color='w', fontsize=12, weight='bold')

        outline_effect = [patheffects.withStroke(linewidth=5, foreground='k')]
        text_time.set_path_effects(outline_effect)
        text_time2.set_path_effects(outline_effect)
    



    # Plot geopotential height at 500 hPa using xarray's contour wrapper
    ax_inset.contour(data['lon'], data["lat"], data['Geopotential_height_isobaric'][Step,21,:,:],
                     levels=np.arange(5100, 6000, 40), colors='k',transform=data_crs,alpha=0.8)



    #Hi_Lo.plot_maxmin_points(ax_inset,data['lon'], data['lat'], data['MSLP_Eta_model_reduction_msl']/100., 'max', 
    #                         50, symbol='H', color='b',  transform=data_crs)

    #Hi_Lo.plot_maxmin_points(ax_inset,data['lon'], data['lat'], data['MSLP_Eta_model_reduction_msl']/100., 'min', 
    #                         25, symbol='L', color='r', transform=data_crs)


    # Plot the path of the cross section
    endpoints = data_crs.transform_points(ccrs.LambertConformal(),
                    *np.vstack([start, end])) #

    #print(endpoints)
    #print(cross['lon'][0],cross['lon'][-1],cross['lat'][-1],cross['lat'][0])
    
    ax_inset.scatter(cross['lon'][0],cross['lat'][0], c='r', zorder=15,transform=data_crs,marker='$S$',s=50)
    ax_inset.scatter(cross['lon'][-1],cross['lat'][-1], c='r', zorder=15,transform=data_crs,marker='$E$',s=50)
    #ax_inset.scatter(259.5,37.25, c='b', zorder=1,transform=data_crs)
    ax_inset.plot(cross['lon'], cross['lat'], c='r', zorder=2,transform=data_crs)

    # Add geographic features
    ax_inset.coastlines()
    ax_inset.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='b', alpha=0.8, zorder=0)

    #extent = [-130,-60,20,52]
    #extent = [start[1]+10,end[1]-10,start[0]-5,end[0]+5]
    if Extent !=None:
        extent = Extent
    else:
        extent = [start[1]+10,end[1]-15,start[0]-10,end[0]+15]
    ax_inset.set_extent(extent,ccrs.PlateCarree())
    # Set the titles and axes labels
    #ax_inset.set_title('NWS Radar (dbz) and 500mb Heights')
    #ax.set_title('NARR Cross-Section \u2013 {} to {} \u2013 Valid: {}\n'
    #             'Potential Temperature (K), Tangential/Normal Winds (knots), '
    #             'Relative Humidity (dimensionless)\n'
    #             'Inset: Cross-Section Path and 500 hPa Geopotential Height'.format(
    #                 start, end, cross['time'].dt.strftime('%Y-%m-%d %H:%MZ').item()))
    ax.set_ylabel('Pressure (Pa)')
    ax.set_xlabel('Longitude (degrees east)')

    time = data.variables["time"]
    print(time.values)

    #type(time.values)
    time.values.tolist()
    time.values[Step].astype('datetime64[s]')
    TIME = time.values[Step].astype('datetime64[s]').tolist()
    #type(TIME)
    file_time = '{0:%Y_%m_%d_%H:%MZ}'.format(TIME)

    ax.set_title("GFS: Theta and RH Cross Section "+"{0:%Y-%m-%d %H:%MZ}".format(TIME))

    fig.savefig("/Users/chowdahead/Desktop/RH_cross"+file_time.replace(":","")+"Z.png",bbox_inches='tight',dpi=120)
    plt.close(fig)
    #plt.show()

# Temperature

In [8]:
def TempCross(File,Year,Month,Day,Step,start,end,Radar=False,Grids=False,inset_dims=None,Extent=None):
    # Define the figure object and primary axes
    fig = plt.figure(1, figsize=(16., 9.))
    ax = plt.axes()

    
    ##############################################################################################################
    # Set up cross section 
    ##############################################################################################################
    
    data = xr.open_dataset(File)
    #data = pg.open('GFS_Global_0p25deg_20190313_0000.grib2.nc')
    #data.read()
    data = data.metpy.parse_cf().squeeze()
    #start = (40, -97.0)
    #end = (36, -93.0)
    cross = cross_section(data, start, end)
    cross.set_coords(('lat', 'lon'), True)

    temperature, pressure = xr.broadcast(cross['Temperature_isobaric'],cross['isobaric4'])

    theta = mpcalc.potential_temperature(pressure, temperature)
    #rh = mpcalc.relative_humidity_from_specific_humidity(specific_humidity, temperature, pressure)

    # These calculations return unit arrays, so put those back into DataArrays in our Dataset
    cross['Potential_temperature'] = xr.DataArray(theta,
                                                  coords=temperature.coords,
                                                  dims=temperature.dims,
                                                  attrs={'units': theta.units})
    #cross['Relative_humidity'] = xr.DataArray(rh,
    #                                          coords=specific_humidity.coords,
    #                                          dims=specific_humidity.dims,
    #                                          attrs={'units': rh.units})

    cross['u-component_of_wind_isobaric'].metpy.convert_units('knots')
    cross['v-component_of_wind_isobaric'].metpy.convert_units('knots')
    cross['t_wind'], cross['n_wind'] = mpcalc.cross_section_components(cross['u-component_of_wind_isobaric'],
                                                                       cross['v-component_of_wind_isobaric'])
    #cross['dew'] =  xr.broadcast(cross['Geopotential_height_potential_vorticity_surface'])
    #print(cross)


    # Plot potential temperature using contour, with some custom labeling
    #theta_contour = ax.contour(lons, cross['isobaric'], cross['Potential_temperature'][0,:,:],
    #                           levels=np.arange(250, 450, 5), colors='k', linewidths=2)

    temp_contour = ax.contourf(cross['lon'], cross['isobaric4'], cross['Temperature_isobaric'][Step,:],
                            cmap="nipy_spectral",levels=np.arange(200, 330, 5))
    cbar = plt.colorbar(temp_contour, orientation='vertical',pad=0.004)#,cax=cbaxes)
    
    theta_contour = ax.contour(cross['lon'], cross['isobaric4'], cross['Potential_temperature'][Step,:,:],
                               levels=np.arange(250, 850, 3), colors='k', linewidths=2)


    # Colorbar Axis Placement (under figure)
    #colorbar_axis = [0.183, 0.09, 0.659, 0.03] # [left, bottom, width, height]
    #cbaxes = fig.add_axes(colorbar_axis)


    # Plot winds using the axes interface directly, with some custom indexing to make the barbs
    # less crowded
    wind_slc_vert = list(range(0, 19, 2)) + list(range(19, 29))
    wind_slc_horz = slice(5, 100, 5)
    #ax.barbs(cross['lon'][wind_slc_horz], cross['isobaric'][wind_slc_vert],
    #         cross['t_wind'][Step,:,:][wind_slc_vert, wind_slc_horz],
    #         cross['n_wind'][Step,:,:][wind_slc_vert, wind_slc_horz], color='b')

    # Adjust the y-axis to be logarithmic
    #ax.set_yscale('symlog')
    plt.yscale('log')
    #ax.set_yticklabels(np.arange(1000, 50, -100))
    #ax.set_ylim(cross['isobaric'].max(), cross['isobaric'].min())
    ax.set_ylim(cross['isobaric'].max(), cross['isobaric'][10])



    ##############################################################################################################
    # Insert
    ##############################################################################################################

    # Define the CRS and inset axes
    data_crs = data['Relative_humidity_isobaric'].metpy.cartopy_crs
    #print(data_crs)
    #4-tuple of floats *rect* = ``[left, bottom, width, height]
    if inset_dims !=None:
        ax_inset = plt.axes(inset_dims, projection=ccrs.LambertConformal())
    else:
        
        ax_inset = plt.axes((0.02, 0.58, 0.23, 0.2), projection=ccrs.LambertConformal())
    #ax_inset = fig.add_axes([0.125, 0.665, 0.25, 0.25], projection=data_crs)

    
    plot_proj = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)
    if Grids == True:
        # *must* call draw in order to get the axis boundary used to add ticks:
        fig.canvas.draw()

        # Define gridline locations and draw the lines using cartopy's built-in gridliner:
        xticks = [np.arange(-180,180)]
        yticks = [np.arange(0,90)]
        ax_inset.gridlines(xlocs=xticks, ylocs=yticks)

        # Label the end-points of the gridlines using the custom tick makers:
        ax_inset.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
        ax_inset.yaxis.set_major_formatter(LATITUDE_FORMATTER)
        #lambert_xticks(ax, xticks)
        #lambert_yticks(ax, yticks)

        #gl = ax_inset.gridlines(draw_labels=True,crs=ccrs.LambertConformal())
        #gl.xlabels_top = gl.ylabels_right = False
        #gl.xformatter = LONGITUDE_FORMATTER
        #gl.yformatter = LATITUDE_FORMATTER
    if Radar == True:
        from siphon.catalog import TDSCatalog
        top_cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog.xml')
        ref = top_cat.catalog_refs['Radar Data']

        new_cat = ref.follow()
        model = new_cat.catalog_refs[4]
        gfs_cat = model.follow()

        ds = gfs_cat.datasets[0]
        subset = ds.subset()
        query_data = subset.query()

        from datetime import datetime, timedelta
        now = datetime(Year,Month,Day,Step*3,0)
        query_data.accept('netcdf4')
        query_data.time(now)
        query_data.variables('Base_reflectivity_surface_layer').add_lonlat()

        radar_data = subset.get_data(query_data)

        time = radar_data.variables['time']
        lon = radar_data.variables['lon']
        lat = radar_data.variables['lat']
        dtime = num2date(time[:],time.units)
        X = radar_data.variables['x'][:]
        Y = radar_data.variables['y'][:]

        Time = dtime[0].strftime('%Y-%m-%d %H:%MZ')
        reflec_data = radar_data.variables['Base_reflectivity_surface_layer']
        reflec = radar_data.variables['Base_reflectivity_surface_layer'][:]
        reflec = np.ma.masked_where(reflec < 9,reflec)

        RADAR_levs = np.arange(0,75,1)

        cs = ax_inset.pcolormesh(lon[:], lat[:], reflec[0,:,:], cmap=mycmap,transform=ccrs.PlateCarree(),vmin=0, vmax=75)
        #cbar = plt.colorbar(cs, orientation='vertical',pad=0.005,aspect=50)
        text_time = ax_inset.text(.995, 0.01, 
                    Time,
                    horizontalalignment='right', transform=ax_inset.transAxes,
                    color='w', fontsize=12, weight='bold')

        text_time2 = ax_inset.text(0.005, 0.01, 
                        "NWS Radar (dbz) & 500mb Heights",
                        horizontalalignment='left', transform=ax_inset.transAxes,
                        color='w', fontsize=12, weight='bold')

        outline_effect = [patheffects.withStroke(linewidth=5, foreground='k')]
        text_time.set_path_effects(outline_effect)
        text_time2.set_path_effects(outline_effect)
    


    # Plot geopotential height at 500 hPa using xarray's contour wrapper
    ax_inset.contour(data['lon'], data["lat"], data['Geopotential_height_isobaric'][Step,21,:,:],
                     levels=np.arange(5100, 6000, 40), colors='k',transform=data_crs,alpha=0.8)


    #Hi_Lo.plot_maxmin_points(ax_inset,data['lon'], data['lat'], data['MSLP_Eta_model_reduction_msl']/100., 'max', 
    #                         50, symbol='H', color='b',  transform=data_crs)

    #Hi_Lo.plot_maxmin_points(ax_inset,data['lon'], data['lat'], data['MSLP_Eta_model_reduction_msl']/100., 'min', 
    #                         25, symbol='L', color='r', transform=data_crs)


    # Plot the path of the cross section
    endpoints = data_crs.transform_points(data_crs,
                    *np.vstack([start, end])) #ccrs.LambertConformal()

    #print(endpoints)
    #print(cross['lon'][0],cross['lon'][-1],cross['lat'][-1],cross['lat'][0])
    
    ax_inset.scatter(cross['lon'][0],cross['lat'][0], c='r', zorder=15,transform=data_crs,marker='$S$',s=50)
    ax_inset.scatter(cross['lon'][-1],cross['lat'][-1], c='r', zorder=15,transform=data_crs,marker='$E$',s=50)
    #ax_inset.scatter(259.5,37.25, c='b', zorder=1,transform=data_crs)
    ax_inset.plot(cross['lon'], cross['lat'], c='r', zorder=2,transform=data_crs)

    # Add geographic features
    ax_inset.coastlines()
    ax_inset.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='b', alpha=0.8, zorder=0)

    #extent = [-130,-60,20,52]
    if Extent !=None:
        extent = Extent
    else:
        extent = [start[1]+10,end[1]-15,start[0]-10,end[0]+15]
    ax_inset.set_extent(extent,ccrs.PlateCarree())
    # Set the titles and axes labels
    #ax_inset.set_title('NWS Radar (dbz) and 500mb Heights')
    #ax.set_title('NARR Cross-Section \u2013 {} to {} \u2013 Valid: {}\n'
    #             'Potential Temperature (K), Tangential/Normal Winds (knots), '
    #             'Relative Humidity (dimensionless)\n'
    #             'Inset: Cross-Section Path and 500 hPa Geopotential Height'.format(
    #                 start, end, cross['time'].dt.strftime('%Y-%m-%d %H:%MZ').item()))
    ax.set_ylabel('Pressure (Pa)')
    ax.set_xlabel('Longitude (degrees east)')

    time = data.variables["time"]
    #print(time.values)

    #type(time.values)
    time.values.tolist()
    time.values[Step].astype('datetime64[s]')
    TIME = time.values[Step].astype('datetime64[s]').tolist()
    #type(TIME)
    file_time = '{0:%Y_%m_%d_%H:%MZ}'.format(TIME)
    print(start[0],start[1],"and",end[0],end[1])
    ax.set_title("GFS: Theta and Temp (K) Cross Section "+'({} N, {} E)'.format(start[0],start[1])+\
                 " to "+'({} N, {} E)'.format(end[0],end[1]),loc="left")
    
    ax.set_title("{0:%Y-%m-%d %H:%MZ}".format(TIME),loc="right")

    fig.savefig("/Users/chowdahead/Desktop/Temp_cross"+file_time.replace(":","")+"Z.png",bbox_inches='tight',dpi=120)
    plt.close(fig)
    #plt.show()

# PV will be difficult because unlike the other variables, it doens't have a vertical coordinate like isobaric4, or something....

In [9]:
print("Temperature_isobaric: Temp\nRelative_humidity_isobaric: RH\nVertical_velocity_pressure_isobaric: Omega\nGeopotential_height_potential_vorticity_surface: PV")

Temperature_isobaric: Temp
Relative_humidity_isobaric: RH
Vertical_velocity_pressure_isobaric: Omega
Geopotential_height_potential_vorticity_surface: PV


In [84]:
def Var_Cross(Variable,Year,Month,Day,Step,start,end,Radar=False,inset=False,Grids=False,
              inset_dims=None,Extent=None,Data=None,File=None):
    
    
    
    # Define the figure object and primary axes
    fig = plt.figure(1, figsize=(16., 9.))
    ax = plt.axes()

    
    
    
    ##############################################################################################################
    # Set up cross section 
    ##############################################################################################################
    #print(Data == None)
    #if Data == None:
    #    print("aha")
    #if Data == None:
    #    data = xr.open_dataset(File)
        #data = pg.open('GFS_Global_0p25deg_20190313_0000.grib2.nc')
        #data.read()
    #    data = data.metpy.parse_cf().squeeze()
        #start = (40, -97.0)
        #end = (36, -93.0)


            ####################################################################
            #time = data.variables["time2"][:]
            #print(time.values,len(time.values))
            #Max = data["Pressure_potential_vorticity_surface"][:].max()
            #Min = data["Pressure_potential_vorticity_surface"][:].min()
            #vert_levs = np.arange(3000,20000,5)
            #data2 = data["Pressure_potential_vorticity_surface"][:,1:,:].squeeze()
            #print(data2.shape)
            #data2 = np.zeros((len(time.values),len(vert_levs),len(data['lat']),len(data['lon'])))
            #data2.shape
            #data2[:] = 2e-06
            #time.values

            #data = xr.DataArray([1, 2, 3], dims='x', coords={'x': [10, 20, 30]})
            #print(data)
            #data_newcoord = data.assign_coords(vert_levs=vert_levs)
            #data = data_newcoord.expand_dims('vert_levels')
            #print(data.)
            #print(data)
            #AHHA = xr.DataArray(data2, coords=[time.values,vert_levs, data['lat'],data['lon']], 
            #             dims=['time2',"vert_levs",'lat','lon'])


            #data["Pressure_potential_vorticity_surface2"]=(['time2', 'vert_levs', 'lat', 'lon'],  AHHA)
            #print("AHHHHHAHHHHHAHAHAHAHAHAH\n",data)


            #data = data.metpy.parse_cf().squeeze()
            ####################################################################
    if Data !=None:
        data = Data
    
    if File !=None:
        data = xr.open_dataset(File)    
        data = data.metpy.parse_cf().squeeze()
        
    if Variable == "Pressure_potential_vorticity_surface":
        Variable = "Pressure_potential_vorticity_surface2"
        var_levs = np.arange(0,3e-06,2e-06)
        print("AHHHHHHH")
        iso_lev = "vert_levs"
        my_cmap = "jet_r"
    
    if Variable == 'Relative_humidity_isobaric':
        var_levs = np.arange(0,120,1)
        iso_lev = 'isobaric'
        my_cmap = "BrBG"
    
    if Variable == 'Temperature_isobaric':
        var_levs = np.arange(200, 330, 5)
        iso_lev = 'isobaric4'
        my_cmap = "nipy_spectral"
        
    if Variable == 'Vertical_velocity_pressure_isobaric':
        var_levs = 30
        iso_lev = 'isobaric6'
        my_cmap = 'PuOr_r'
    
    var_dict = {'Temperature_isobaric':"Temp",'Relative_humidity_isobaric':"RH",
                'Vertical_velocity_pressure_isobaric':"Omega",
               "Pressure_potential_vorticity_surface2":"PV"}
    
    
    
    
    
    
    cross = cross_section(data, start, end)
    cross.set_coords(('lat', 'lon'), True)
    #print(cross['isobaric'])
    temperature, pressure = xr.broadcast(cross['Temperature_isobaric'],cross['isobaric4'])

    theta = mpcalc.potential_temperature(pressure, temperature)
    #rh = mpcalc.relative_humidity_from_specific_humidity(specific_humidity, temperature, pressure)

    # These calculations return unit arrays, so put those back into DataArrays in our Dataset
    cross['Potential_temperature'] = xr.DataArray(theta,
                                                  coords=temperature.coords,
                                                  dims=temperature.dims,
                                                  attrs={'units': theta.units})
    #cross['Relative_humidity'] = xr.DataArray(rh,
    #                                          coords=specific_humidity.coords,
    #                                          dims=specific_humidity.dims,
    #                                          attrs={'units': rh.units})

    cross['u-component_of_wind_isobaric'].metpy.convert_units('knots')
    cross['v-component_of_wind_isobaric'].metpy.convert_units('knots')
    cross['t_wind'], cross['n_wind'] = mpcalc.cross_section_components(cross['u-component_of_wind_isobaric'],
                                                                       cross['v-component_of_wind_isobaric'])
    #cross['dew'] =  xr.broadcast(cross['Geopotential_height_potential_vorticity_surface'])
    #print(cross)


    # Plot potential temperature using contour, with some custom labeling
    #theta_contour = ax.contour(lons, cross['isobaric'], cross['Potential_temperature'][0,:,:],
    #                           levels=np.arange(250, 450, 5), colors='k', linewidths=2)
    
    print(iso_lev)
    if Variable == "Pressure_potential_vorticity_surface2":
        contour = ax.contour(cross['lon'], cross[iso_lev], cross[Variable][Step,:],
                            cmap=my_cmap,levels=var_levs,alpha=0.7)
        
    else:
        contour = ax.contourf(cross['lon'], cross[iso_lev], cross[Variable][Step,:],
                            cmap=my_cmap,levels=var_levs,alpha=0.7)
        cbar = plt.colorbar(contour, orientation='vertical',pad=0.004)#,cax=cbaxes)
    
    #'isobaric4'
    theta_contour = ax.contour(cross['lon'], cross['isobaric4'], cross['Potential_temperature'][Step,:,:],
                               levels=np.arange(250, 850, 3), colors='k', linewidths=2)


    # Colorbar Axis Placement (under figure)
    #colorbar_axis = [0.183, 0.09, 0.659, 0.03] # [left, bottom, width, height]
    #cbaxes = fig.add_axes(colorbar_axis)


    # Plot winds using the axes interface directly, with some custom indexing to make the barbs
    # less crowded
    wind_slc_vert = list(range(0, 19, 2)) + list(range(19, 29))
    wind_slc_horz = slice(5, 100, 5)
    #ax.barbs(cross['lon'][wind_slc_horz], cross['isobaric'][wind_slc_vert],
    #         cross['t_wind'][Step,:,:][wind_slc_vert, wind_slc_horz],
    #         cross['n_wind'][Step,:,:][wind_slc_vert, wind_slc_horz], color='b')

    # Adjust the y-axis to be logarithmic
    #ax.set_yscale('symlog')
    plt.yscale('log')
    #ax.set_yticklabels(np.arange(1000, 50, -100))
    #ax.set_ylim(cross['isobaric'].max(), cross['isobaric'].min())
    ax.set_ylim(cross['isobaric'].max(), cross['isobaric'][10])


    if inset == True:
    ##############################################################################################################
    # Insert
    ##############################################################################################################

        # Define the CRS and inset axes
        data_crs = data[Variable].metpy.cartopy_crs
        print(data_crs)
        #4-tuple of floats *rect* = ``[left, bottom, width, height]
        if inset_dims !=None:
            ax_inset = plt.axes(inset_dims, projection=ccrs.LambertConformal())
        else:

            ax_inset = plt.axes((0.02, 0.58, 0.23, 0.2), projection=ccrs.LambertConformal())
        #ax_inset = fig.add_axes([0.125, 0.665, 0.25, 0.25], projection=data_crs)

        plot_proj = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)
        if Grids == True:
            # *must* call draw in order to get the axis boundary used to add ticks:
            fig.canvas.draw()

            # Define gridline locations and draw the lines using cartopy's built-in gridliner:
            xticks = [np.arange(-180,180)]
            yticks = [np.arange(0,90)]
            ax_inset.gridlines(xlocs=xticks, ylocs=yticks)

            # Label the end-points of the gridlines using the custom tick makers:
            ax_inset.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
            ax_inset.yaxis.set_major_formatter(LATITUDE_FORMATTER)
            #lambert_xticks(ax, xticks)
            #lambert_yticks(ax, yticks)

            #gl = ax_inset.gridlines(draw_labels=True,crs=ccrs.LambertConformal())
            #gl.xlabels_top = gl.ylabels_right = False
            #gl.xformatter = LONGITUDE_FORMATTER
            #gl.yformatter = LATITUDE_FORMATTER
            
            
            # Plot geopotential height at 500 hPa using xarray's contour wrapper
            ax_inset.contour(data['lon'], data["lat"], data['Geopotential_height_isobaric'][Step,21,:,:],
                             levels=np.arange(5100, 6000, 40), colors='k',transform=data_crs,alpha=0.8)



            # Plot the path of the cross section
            endpoints = data_crs.transform_points(ccrs.LambertConformal(),
                            *np.vstack([start, end])) #

            #print(endpoints)
            #print(cross['lon'][0],cross['lon'][-1],cross['lat'][-1],cross['lat'][0])

            ax_inset.scatter(cross['lon'][0],cross['lat'][0], c='r', zorder=15,transform=data_crs,marker='$S$',s=50)
            ax_inset.scatter(cross['lon'][-1],cross['lat'][-1], c='r', zorder=15,transform=data_crs,marker='$E$',s=50)
            #ax_inset.scatter(259.5,37.25, c='b', zorder=1,transform=data_crs)
            ax_inset.plot(cross['lon'], cross['lat'], c='r', zorder=2,transform=data_crs)

            # Add geographic features
            ax_inset.coastlines()
            ax_inset.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='b', alpha=0.8, zorder=0)

            #extent = [-130,-60,20,52]
            #extent = [start[1]+10,end[1]-10,start[0]-5,end[0]+5]
            if Extent !=None:
                extent = Extent
            else:
                extent = [start[1]+10,end[1]-15,start[0]-10,end[0]+15]
            ax_inset.set_extent(extent,ccrs.PlateCarree())

    if Radar == True:
        from siphon.catalog import TDSCatalog
        top_cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog.xml')
        ref = top_cat.catalog_refs['Radar Data']

        new_cat = ref.follow()
        model = new_cat.catalog_refs[4]
        gfs_cat = model.follow()

        ds = gfs_cat.datasets[0]
        subset = ds.subset()
        query_data = subset.query()

        from datetime import datetime, timedelta
        now = datetime(Year,Month,Day,Step*3,0)
        query_data.accept('netcdf4')
        query_data.time(now)
        query_data.variables('Base_reflectivity_surface_layer').add_lonlat()

        radar_data = subset.get_data(query_data)

        time = radar_data.variables['time']
        lon = radar_data.variables['lon']
        lat = radar_data.variables['lat']
        dtime = num2date(time[:],time.units)
        X = radar_data.variables['x'][:]
        Y = radar_data.variables['y'][:]

        Time = dtime[0].strftime('%Y-%m-%d %H:%MZ')
        reflec_data = radar_data.variables['Base_reflectivity_surface_layer']
        reflec = radar_data.variables['Base_reflectivity_surface_layer'][:]
        reflec = np.ma.masked_where(reflec < 9,reflec)

        RADAR_levs = np.arange(0,75,1)
    
        cs = ax_inset.pcolormesh(lon[:], lat[:], reflec[0,:,:], cmap=mycmap,transform=ccrs.PlateCarree(),vmin=0, vmax=75)
        #cbar = plt.colorbar(cs, orientation='vertical',pad=0.005,aspect=50)
        text_time = ax_inset.text(.995, 0.01, 
                    Time,
                    horizontalalignment='right', transform=ax_inset.transAxes,
                    color='w', fontsize=12, weight='bold')

        text_time2 = ax_inset.text(0.005, 0.01, 
                        "NWS Radar (dbz) & 500mb Heights",
                        horizontalalignment='left', transform=ax_inset.transAxes,
                        color='w', fontsize=12, weight='bold')

        outline_effect = [patheffects.withStroke(linewidth=5, foreground='k')]
        text_time.set_path_effects(outline_effect)
        text_time2.set_path_effects(outline_effect)
    



    #Hi_Lo.plot_maxmin_points(ax_inset,data['lon'], data['lat'], data['MSLP_Eta_model_reduction_msl']/100., 'max', 
    #                         50, symbol='H', color='b',  transform=data_crs)

    #Hi_Lo.plot_maxmin_points(ax_inset,data['lon'], data['lat'], data['MSLP_Eta_model_reduction_msl']/100., 'min', 
    #                         25, symbol='L', color='r', transform=data_crs)

        # Set the titles and axes labels
    #ax_inset.set_title('NWS Radar (dbz) and 500mb Heights')
    #ax.set_title('NARR Cross-Section \u2013 {} to {} \u2013 Valid: {}\n'
    #             'Potential Temperature (K), Tangential/Normal Winds (knots), '
    #             'Relative Humidity (dimensionless)\n'
    #             'Inset: Cross-Section Path and 500 hPa Geopotential Height'.format(
    #                 start, end, cross['time'].dt.strftime('%Y-%m-%d %H:%MZ').item()))
    ax.set_ylabel('Pressure (Pa)')
    ax.set_xlabel('Longitude (degrees east)')

    time = data.variables["time2"]
    print(time.values)

    #type(time.values)
    time.values.tolist()
    time.values[Step].astype('datetime64[s]')
    TIME = time.values[Step].astype('datetime64[s]').tolist()
    #type(TIME)
    file_time = '{0:%Y_%m_%d_%H:%MZ}'.format(TIME)

    ax.set_title("GFS: Theta and "+var_dict[Variable]+" Cross Section "+"{0:%Y-%m-%d %H:%MZ}".format(TIME))

    fig.savefig("/Users/chowdahead/Desktop/"+var_dict[Variable]+"_cross"+file_time.replace(":","")+"Z.png",bbox_inches='tight',dpi=120)
    plt.close(fig)
    #plt.show()

In [87]:
File = 'GFS_Global_0p25deg_20190916_1200.grib2.nc'
#File = 'GFS_Global_0p5deg_20190522_0000.grib2.nc'
File = "GFS_Global_0p25deg_20190916_1200.grib2.nc"
data = xr.open_dataset(File)

data = data.metpy.parse_cf().squeeze()

#for var in data:
#    print(var)
#type(data)
#data

In [88]:
list(data.variables)

['time2',
 'lat',
 'lon',
 'crs',
 'isobaric',
 'isobaric4',
 'isobaric6',
 'potential_vorticity_surface',
 'MSLP_Eta_model_reduction_msl',
 'LatLon_Projection',
 'Relative_humidity_isobaric',
 'u-component_of_wind_isobaric',
 'v-component_of_wind_isobaric',
 'Geopotential_height_isobaric',
 'Temperature_isobaric',
 'Vertical_velocity_pressure_isobaric',
 'Pressure_potential_vorticity_surface']

In [89]:
time = data.variables["time2"][:]
print(time.values,len(time.values))


['2019-09-16T15:00:00.000000000' '2019-09-16T18:00:00.000000000'] 2


In [90]:
vert_levs = np.arange(20000,30000,1)

In [91]:


data2 = data["Pressure_potential_vorticity_surface"][:,1,:,:].squeeze()
print(data2.shape)
data2 = np.zeros((len(time.values),len(vert_levs),len(data['lat']),len(data['lon'])))
data2.shape
#data2[:] = 2e-06
#time.values

(2, 141, 241)


(2, 10000, 141, 241)

In [92]:
start = (45, -110.0)
end = (35, -97.0)

inset = (0.0255, 0.45, 0.43, 0.43)
extent = [start[1]-5,end[1]+5,start[0]+5,end[0]-5]

print(np.where(data['lon'].values==360+start[1]),np.where(data['lat'].values==end[0]))
print(np.where(data['lon'].values==360+end[1]),np.where(data['lat'].values==start[0]))
lon_start_index = np.where(data['lon'].values==360+start[1])
lon_end_index = np.where(data['lon'].values==360+end[1])
lat_start_index = np.where(data['lat'].values==start[0])
lat_end_index = np.where(data['lat'].values==end[0])
lat_end_index[0][0]
print("Lat start:",lat_start_index[0][0],"end:",lat_end_index[0][0],
      "\nLon start:",lon_start_index[0][0],"end:",lon_end_index[0][0])

(array([80]),) (array([80]),)
(array([132]),) (array([40]),)
Lat start: 40 end: 80 
Lon start: 80 end: 132


In [ ]:
%%time
for m in range(0,1):
    for l in range(lat_start_index[0][0],lat_end_index[0][0]-15+1):
        for k in range(lon_start_index[0][0],lon_end_index[0][0]-30+1):
            for j in range(len(vert_levs)): 
                if vert_levs[j] == data["Pressure_potential_vorticity_surface"][m,1,l,k].values.astype(int):
                    print("ahh:",vert_levs[j],m,l,k)
                    data2[m,1,l,k] = 2e-06

ahh: 21842 0 40 93


In [67]:
data_newcoord = data.assign_coords(vert_levs=vert_levs)
data = data_newcoord.expand_dims('vert_levels')
#print(data.)
print(data)

<xarray.Dataset>
Dimensions:                               (isobaric: 31, isobaric4: 34, isobaric6: 21, lat: 141, lon: 241, potential_vorticity_surface: 2, time2: 2, vert_levels: 1, vert_levs: 500)
Coordinates:
  * time2                                 (time2) datetime64[ns] 2019-09-16T15:00:00 2019-09-16T18:00:00
  * lat                                   (lat) float32 55.0 54.75 ... 20.0
  * lon                                   (lon) float32 230.0 230.25 ... 290.0
    crs                                   object Projection: latitude_longitude
  * isobaric                              (isobaric) float32 100.0 ... 100000.0
  * isobaric4                             (isobaric4) float32 40.0 ... 100000.0
  * isobaric6                             (isobaric6) float32 10000.0 ... 100000.0
  * potential_vorticity_surface           (potential_vorticity_surface) float32 -2e-06 2e-06
  * vert_levs                             (vert_levs) int64 20000 ... 20499
Dimensions without coordinates: vert_

In [68]:
AHHA = xr.DataArray(data2, coords=[time.values,vert_levs, data['lat'],data['lon']], 
             dims=['time2',"vert_levs",'lat','lon'])

In [69]:
data["Pressure_potential_vorticity_surface2"]=(['time2', 'vert_levs', 'lat', 'lon'],  AHHA)
data = data.metpy.parse_cf().squeeze()
print(data)

<xarray.Dataset>
Dimensions:                                (isobaric: 31, isobaric4: 34, isobaric6: 21, lat: 141, lon: 241, potential_vorticity_surface: 2, time2: 2, vert_levs: 500)
Coordinates:
  * time2                                  (time2) datetime64[ns] 2019-09-16T15:00:00 2019-09-16T18:00:00
  * lat                                    (lat) float32 55.0 54.75 ... 20.0
  * lon                                    (lon) float32 230.0 230.25 ... 290.0
    crs                                    object Projection: latitude_longitude
  * isobaric                               (isobaric) float32 100.0 ... 100000.0
  * isobaric4                              (isobaric4) float32 40.0 ... 100000.0
  * isobaric6                              (isobaric6) float32 10000.0 ... 100000.0
  * potential_vorticity_surface            (potential_vorticity_surface) float32 -2e-06 2e-06
  * vert_levs                              (vert_levs) int64 20000 ... 20499
Data variables:
    MSLP_Eta_model_reductio

In [52]:
File

'GFS_Global_0p25deg_20190916_1200.grib2.nc'

In [85]:
%%time

Var_Cross("Pressure_potential_vorticity_surface",
                2019,9,16,0,start,end,Radar=False,inset=False,inset_dims=inset,Extent=extent,Data=data) #Data=data File=File

AHHHHHHH


/Users/chowdahead/anaconda3/envs/test-grib/lib/python3.7/site-packages/ipykernel_launcher.py:96: FutureWarning: The inplace argument has been deprecated and will be removed in a future version of xarray.
/Users/chowdahead/anaconda3/envs/test-grib/lib/python3.7/site-packages/ipykernel_launcher.py:128: UserWarning: No contour levels were found within the data range.


vert_levs
['2019-09-16T15:00:00.000000000' '2019-09-16T18:00:00.000000000']
CPU times: user 3.62 s, sys: 274 ms, total: 3.89 s
Wall time: 3.94 s


In [47]:
data['Pressure_potential_vorticity_surface2']

<xarray.DataArray 'Pressure_potential_vorticity_surface2' (time2: 2, vert_levs: 500, lat: 141, lon: 241)>
array([[[[0., ..., 0.],
         ...,
         [0., ..., 0.]],

        ...,

        [[0., ..., 0.],
         ...,
         [0., ..., 0.]]],


       [[[0., ..., 0.],
         ...,
         [0., ..., 0.]],

        ...,

        [[0., ..., 0.],
         ...,
         [0., ..., 0.]]]])
Coordinates:
  * time2      (time2) datetime64[ns] 2019-09-16T15:00:00 2019-09-16T18:00:00
  * lat        (lat) float32 55.0 54.75 54.5 54.25 ... 20.75 20.5 20.25 20.0
  * lon        (lon) float32 230.0 230.25 230.5 230.75 ... 289.5 289.75 290.0
    crs        object Projection: latitude_longitude
  * vert_levs  (vert_levs) int64 20000 20001 20002 20003 ... 20497 20498 20499

In [57]:
data['Vertical_velocity_pressure_isobaric']

<xarray.DataArray 'Vertical_velocity_pressure_isobaric' (time2: 2, isobaric6: 21, lat: 141, lon: 241)>
[1427202 values with dtype=float32]
Coordinates:
  * time2      (time2) datetime64[ns] 2019-09-16T15:00:00 2019-09-16T18:00:00
  * lat        (lat) float32 55.0 54.75 54.5 54.25 ... 20.75 20.5 20.25 20.0
  * lon        (lon) float32 230.0 230.25 230.5 230.75 ... 289.5 289.75 290.0
    crs        object Projection: latitude_longitude
  * isobaric6  (isobaric6) float32 10000.0 15000.0 20000.0 ... 97500.0 100000.0
Attributes:
    long_name:                      Vertical velocity (pressure) @ Isobaric s...
    units:                          Pa/s
    abbreviation:                   VVEL
    grid_mapping:                   LatLon_Projection
    Grib_Variable_Id:               VAR_0-2-8_L100
    Grib2_Parameter:                [0 2 8]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Momentum
    Grib2_Parameter_Name:           Vertical velocity

In [48]:
data

<xarray.Dataset>
Dimensions:                                (isobaric: 31, isobaric4: 34, isobaric6: 21, lat: 141, lon: 241, potential_vorticity_surface: 2, time2: 2, vert_levs: 500)
Coordinates:
  * time2                                  (time2) datetime64[ns] 2019-09-16T15:00:00 2019-09-16T18:00:00
  * lat                                    (lat) float32 55.0 54.75 ... 20.0
  * lon                                    (lon) float32 230.0 230.25 ... 290.0
    crs                                    object Projection: latitude_longitude
  * isobaric                               (isobaric) float32 100.0 ... 100000.0
  * isobaric4                              (isobaric4) float32 40.0 ... 100000.0
  * isobaric6                              (isobaric6) float32 10000.0 ... 100000.0
  * potential_vorticity_surface            (potential_vorticity_surface) float32 -2e-06 2e-06
  * vert_levs                              (vert_levs) int64 20000 ... 20499
Data variables:
    MSLP_Eta_model_reductio